## 미니 프로젝트 1

### 0. 데이터 불러오기 및 확인

In [1]:
import pandas as pd

df1 = pd.read_csv('origin_data/일별시간대별버스승차인원수현황.csv')
df1

C:\Users\LG\AppData\Local\Temp\ipykernel_21416\254045981.py:3: DtypeWarning: Columns (0,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('origin_data/일별시간대별버스승차인원수현황.csv')


,기준일,요일,시간대,이용자 유형,이용자 수
0,20190101,화,22시-24시,경로,2
1,20190101,화,22시-24시,어린이,1
2,20190101,화,22시-24시,일반,25
3,20190101,화,22시-24시,장애 동반,1
4,20190101,화,22시-24시,청소년,5
...,...,...,...,...,...
272207,20241006,일,22,청소년,125
272208,20241006,일,23,경로,1
272209,20241006,일,23,일반,33
272210,20241006,일,23,장애 일반,1


In [2]:
df2 = pd.read_csv('origin_data/일별버스승차인원수현황.csv')
df2

,base_date,act_type,bus_number,user_type,user_count
0,20190101,승차,101-1,어린이,1
1,20190101,승차,101-1,일반,53
2,20190101,승차,101-1,청소년,15
3,20190101,승차,101-2,일반,46
4,20190101,승차,101-2,청소년,7
...,...,...,...,...,...
11997303,20241006,환승,820-6,일반,2
11997304,20241006,환승,880-1,일반,2
11997305,20241006,환승,880-1,청소년,1
11997306,20241006,환승,999-9,일반,2


In [3]:
df3 = pd.read_csv('origin_data/제주특별자치도_버스정류소현황_20221101.csv', encoding='cp949')
df3

,정류소아이디,정류소명,경도,위도,위치정보(주변설명),데이터 기준일자
0,406001167.0,(구)구판장,126.798051,33.329675,세화1리노인회관입구,2022-11-01
1,406001168.0,(구)구판장,126.797879,33.329732,세화1리노인회관입구,2022-11-01
2,405000186.0,(구)삼양검문소,126.600034,33.521022,자연농원 우측,2022-11-01
3,405000185.0,(구)삼양검문소,126.600847,33.520956,조천읍이정표 좌측,2022-11-01
4,406000324.0,(구)중앙파출소,126.560894,33.247598,윤성현내과의원 건너편,2022-11-01
...,...,...,...,...,...,...
4075,405001116.0,흥국사,126.378405,33.452615,납읍방향.용흥3길 입구,2022-11-01
4076,406001766.0,흥덕사,126.876066,33.399128,NaN,2022-11-01
4077,406001767.0,흥덕사,126.876005,33.399031,NaN,2022-11-01
4078,406001055.0,희진주유소,126.873644,33.376610,희진주유소 건너편,2022-11-01


In [4]:
df4 = pd.read_csv('origin_data/이용자유형별버스정류소이용인원현황.csv')
df4

,base_date,station_name,user_type,user_count
0,20190101,(구)동홍동주민센터 ...,경로,3
1,20190101,(구)동홍동주민센터 ...,어린이,2
2,20190101,(구)동홍동주민센터 ...,일반,14
3,20190101,(구)동홍동주민센터 ...,장애 동반,1
4,20190101,(구)동홍동주민센터 ...,장애 일반,1
...,...,...,...,...
13249590,20241006,흙통 ...,청소년,3
13249591,20241006,흥국사 ...,경로,2
13249592,20241006,흥국사 ...,일반,5
13249593,20241006,희진주유소 ...,경로,1


In [5]:
# 2. df3 컬럼명을 영어로 변경
df3 = df3.rename(columns={
    '정류소아이디': 'station_id',
    '정류소명': 'station_name',
    '경도': 'longitude',
    '위도': 'latitude',
    '위치정보(주변설명)': 'location_info',
    '데이터 기준일자': 'base_date'
})

In [6]:
import unicodedata

def norm_name(s: pd.Series) -> pd.Series:
    s = s.astype(str)
    # 유니코드/공백 정규화
    s = s.map(lambda x: unicodedata.normalize('NFC', x)).str.replace(r'\s+', ' ', regex=True).str.strip()
    return s

df_total = (df4.groupby(['base_date', 'station_name'], as_index=False)['user_count'] .sum() .rename(columns={'user_count': 'total_user_count'}) )

# df3에서 base_date 제거
df3_m2m = df3.drop(columns=['base_date'], errors='ignore').copy()
df3_m2m['station_name_norm'] = norm_name(df3_m2m['station_name'])
df_total_m2m = df_total.copy()
df_total_m2m['station_name_norm'] = norm_name(df_total_m2m['station_name'])

# 많은-대-많이 조인
df5 = (
    df_total_m2m
    .merge(df3_m2m[['station_name_norm','station_id','longitude','latitude','location_info']], 
           on='station_name_norm', how='left')
    .drop(columns=['station_name_norm'])
)

print(df5.head())

   base_date                                       station_name  \
0   20190101  (구)동홍동주민센터                                    ...   
1   20190101  (구)삼일금고                                       ...   
2   20190101  (구)중앙파출소                                      ...   
3   20190101  (구)중앙파출소                                      ...   
4   20190101  (구)중앙파출소                                      ...   

   total_user_count   station_id   longitude   latitude location_info  
0                22          NaN         NaN        NaN           NaN  
1                75          NaN         NaN        NaN           NaN  
2               102  406000324.0  126.560894  33.247598   윤성현내과의원 건너편  
3               102  406000467.0  126.560367  33.247000  M-STAY제주호텔 앞  
4               102  406001176.0  126.560890  33.247515   에이스모텔(후문) 앞  


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272212 entries, 0 to 272211
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   기준일     272212 non-null  object
 1   요일      272212 non-null  object
 2   시간대     272212 non-null  object
 3   이용자 유형  272212 non-null  object
 4   이용자 수   272212 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11997308 entries, 0 to 11997307
Data columns (total 5 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   base_date   int64 
 1   act_type    object
 2   bus_number  object
 3   user_type   object
 4   user_count  int64 
dtypes: int64(2), object(3)
memory usage: 457.7+ MB


In [9]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6294126 entries, 0 to 6294125
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   base_date         int64  
 1   station_name      object 
 2   total_user_count  int64  
 3   station_id        float64
 4   longitude         float64
 5   latitude          float64
 6   location_info     object 
dtypes: float64(3), int64(2), object(2)
memory usage: 336.1+ MB


In [10]:
print(df1.isnull().sum())
print(df2.isnull().sum())
print(df5.isnull().sum())

기준일       0
요일        0
시간대       0
이용자 유형    0
이용자 수     0
dtype: int64
base_date     0
act_type      0
bus_number    0
user_type     0
user_count    0
dtype: int64
base_date                 0
station_name              0
total_user_count          0
station_id           680219
longitude            680219
latitude             680219
location_info       1983501
dtype: int64


In [11]:
df5.shape

(6294126, 7)

### 1. 데이터 정제

In [12]:
# 1. 위치정보(주변설명)을 제외한 나머지 컬럼에서 결측치가 있는 행 삭제
df5 = df5.dropna(subset=['station_id', 'station_name', 'longitude', 'latitude'])

# 결과 확인
print(df5.head())
print(df5.isnull().sum())

   base_date                                       station_name  \
2   20190101  (구)중앙파출소                                      ...   
3   20190101  (구)중앙파출소                                      ...   
4   20190101  (구)중앙파출소                                      ...   
5   20190101  1100고지휴게소                                     ...   
6   20190101  1100고지휴게소                                     ...   

   total_user_count   station_id   longitude   latitude  location_info  
2               102  406000324.0  126.560894  33.247598    윤성현내과의원 건너편  
3               102  406000467.0  126.560367  33.247000   M-STAY제주호텔 앞  
4               102  406001176.0  126.560890  33.247515    에이스모텔(후문) 앞  
5                54  406001894.0  126.462942  33.357820            NaN  
6                54  406001109.0  126.463076  33.357816  1100고지휴게소 맞은편  
base_date                 0
station_name              0
total_user_count          0
station_id                0
longitude                 0
latitude           

In [13]:
df5.shape

(5613907, 7)

In [14]:
# df3 위치정보(주변설명) 결측치를 "정보없음"으로 대체
df5['location_info'] = df5['location_info'].fillna('정보없음')
df5_clean = df5.copy()

df5.isnull().sum()

base_date           0
station_name        0
total_user_count    0
station_id          0
longitude           0
latitude            0
location_info       0
dtype: int64

In [15]:
import numpy as np
from datetime import datetime
import random

# df1 컬럼명 정리 (한글이 깨져서 표시되므로 영어로 변경)
df1_clean = df1.copy()
rename_map = {
    '기준일': 'base_date',
    '요일': 'day_of_week',
    '시간대': 'time_slot',
    '이용자 유형': 'user_type',
    '이용자 수': 'user_count'
}
df1_clean = df1_clean.rename(columns=rename_map)

# df2는 이미 영어 컬럼명이므로 그대로 사용
df2_clean = df2.copy()

print("df1 shape:", df1_clean.shape)
print("df2 shape:", df2_clean.shape)

df1 shape: (272212, 5)
df2 shape: (11997308, 5)


### 2. 데이터 타입 변환

In [16]:
import pandas as pd
import re

# -------------------------------
# 1) 진단: 형식이 이상한 값 찍어보기
# -------------------------------
def preview_bad_dates(s):
    s_str = s.astype(str).str.strip()
    bad_mask = ~s_str.str.fullmatch(r'\d{8}')  # 정확히 8자리 숫자만 통과
    print("[진단] 비정상 base_date 샘플(최대 10개):")
    print(s_str[bad_mask].head(10))

print("=== df1 base_date 진단 전 ===")
preview_bad_dates(df1_clean['base_date'])

print("=== df2 base_date 진단 전 ===")
preview_bad_dates(df2_clean['base_date'])

# -----------------------------------------------------
# 2) 정제: 숫자만 남기고 길이 8 아닌 값은 NaN 처리(추후 NaT)
#   - 'base_date', '2019-01-01', ' 20190101 ', '20190101\n' 등 케이스 흡수
# -----------------------------------------------------
def sanitize_yyyymmdd(series):
    s = series.astype(str).str.strip()
    # 제어문자/BOM 제거
    s = s.str.replace(r'[\uFEFF\u200B-\u200D\u2060\u00A0]', '', regex=True)
    # 숫자만 추출
    s = s.str.replace(r'\D', '', regex=True)
    # 길이 8만 남기고 나머지는 NaN
    s = s.where(s.str.len() == 8, other=pd.NA)
    return s

df1_dates = sanitize_yyyymmdd(df1_clean['base_date'])
df2_dates = sanitize_yyyymmdd(df2_clean['base_date'])

# -----------------------------------------------------
# 3) 안전 변환: YYYYMMDD로만 to_datetime, 나머지는 NaT
# -----------------------------------------------------
df1_clean['base_date'] = pd.to_datetime(df1_dates, format='%Y%m%d', errors='coerce')
df2_clean['base_date'] = pd.to_datetime(df2_dates, format='%Y%m%d', errors='coerce')

# 결과 점검
print("[결과] df1 base_date 변환 실패(NaT) 건수:", df1_clean['base_date'].isna().sum())
print("[결과] df2 base_date 변환 실패(NaT) 건수:", df2_clean['base_date'].isna().sum())

# -----------------------------------------------------
# 4) user_count 숫자화(쉼표/공백/문자 제거 후 변환)
# -----------------------------------------------------
def to_int_clean(series):
    s = series.astype(str).str.strip()
    s = s.str.replace(',', '', regex=False)
    s = s.str.replace(r'[^\d\-+.]', '', regex=True)  # 숫자/부호/점 외 제거
    return pd.to_numeric(s, errors='coerce')

df1_clean['user_count'] = to_int_clean(df1_clean['user_count']).astype('Int64')

print("df1 dtypes:\n", df1_clean.dtypes)
print("df2 dtypes:\n", df2_clean.dtypes)

# -----------------------------------------------------
# (옵션) 변환 실패한 행들 살펴보기
# -----------------------------------------------------
bad_df1 = df1_clean[df1_clean['base_date'].isna()].head(10)
bad_df2 = df2_clean[df2_clean['base_date'].isna()].head(10)
print("[참고] df1 변환 실패 행 샘플:\n", bad_df1)
print("[참고] df2 변환 실패 행 샘플:\n", bad_df2)

=== df1 base_date 진단 전 ===
[진단] 비정상 base_date 샘플(최대 10개):
232304    base_date
Name: base_date, dtype: object
=== df2 base_date 진단 전 ===
[진단] 비정상 base_date 샘플(최대 10개):
Series([], Name: base_date, dtype: object)
[결과] df1 base_date 변환 실패(NaT) 건수: 1
[결과] df2 base_date 변환 실패(NaT) 건수: 0
df1 dtypes:
 base_date      datetime64[ns]
day_of_week            object
time_slot              object
user_type              object
user_count              Int64
dtype: object
df2 dtypes:
 base_date     datetime64[ns]
act_type              object
bus_number            object
user_type             object
user_count             int64
dtype: object
[참고] df1 변환 실패 행 샘플:
        base_date  day_of_week  time_slot  user_type  user_count
232304       NaT  day_of_week  base_hour  user_type        <NA>
[참고] df2 변환 실패 행 샘플:
 Empty DataFrame
Columns: [base_date, act_type, bus_number, user_type, user_count]
Index: []


In [17]:
# 1) 문제 행(헤더가 끼어들어 NaT가 된 행) 제거
n_bad = df1_clean['base_date'].isna().sum()
print(f"[정리 전] df1 base_date NaT 행: {n_bad}")

df1_clean = df1_clean.dropna(subset=['base_date']).reset_index(drop=True)

print(f"[정리 후] df1 shape: {df1_clean.shape}")
print(f"[확인] df1 base_date NaT 행: {df1_clean['base_date'].isna().sum()}")

# 2) user_count 정수형 유지(혹시 모를 float 캐스팅 방지)
df1_clean['user_count'] = df1_clean['user_count'].astype('Int64')

# # 3) 시간대 정규화 → hour 파생(이미 했더라도 재실행해도 무해)
# import re
# def norm_time_slot(x: str):
#     s = str(x).strip()
#     s = s.replace('시', '').replace('−', '-').replace('–', '-')
#     try:
#         if '-' in s:               # '22-24' 같은 범위면 시작시(hour) 사용
#             return int(s.split('-')[0])
#         return int(re.findall(r'\d+', s)[0])  # '22' / '23시' 등
#     except Exception:
#         return pd.NA

# df1_clean['hour'] = df1_clean['time_slot'].apply(norm_time_slot).astype('Int64')

# # 4) 요일 정규화(옵션) — 월(0)~일(6)로 매핑
# weekday_map_kr = {'월':0,'화':1,'수':2,'목':3,'금':4,'토':5,'일':6}
# df1_clean['weekday_num'] = df1_clean['day_of_week'].map(weekday_map_kr).astype('Int64')

# 5) 최종 점검
print(df1_clean.dtypes)
print(df1_clean.head(3))

[정리 전] df1 base_date NaT 행: 1
[정리 후] df1 shape: (272211, 5)
[확인] df1 base_date NaT 행: 0
base_date      datetime64[ns]
day_of_week            object
time_slot              object
user_type              object
user_count              Int64
dtype: object
   base_date day_of_week time_slot user_type  user_count
0 2019-01-01           화   22시-24시        경로           2
1 2019-01-01           화   22시-24시       어린이           1
2 2019-01-01           화   22시-24시        일반          25


In [18]:
# df2의 base_date를 datetime으로 변환
df2_clean['base_date'] = pd.to_datetime(df2_clean['base_date'], format='%Y%m%d')

print("df2 dtypes:", df2_clean.dtypes)

df2 dtypes: base_date     datetime64[ns]
act_type              object
bus_number            object
user_type             object
user_count             int64
dtype: object


In [19]:
df1_clean.columns

Index(['base_date', 'day_of_week', 'time_slot', 'user_type', 'user_count'], dtype='object')

In [20]:
df5['base_date'] = pd.to_datetime(df5['base_date'], format='%Y%m%d')
print(df5.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5613907 entries, 2 to 6294125
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   base_date         datetime64[ns]
 1   station_name      object        
 2   total_user_count  int64         
 3   station_id        float64       
 4   longitude         float64       
 5   latitude          float64       
 6   location_info     object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 342.6+ MB
None


In [21]:
df5_clean = df5.copy()

### 3. 시간 관련 파생 변수 생성

In [22]:
# df1에 시간 파생 변수 추가
df1_clean['year'] = df1_clean['base_date'].dt.year
df1_clean['month'] = df1_clean['base_date'].dt.month
df1_clean['day'] = df1_clean['base_date'].dt.day
df1_clean['weekday'] = df1_clean['base_date'].dt.weekday  # 0: 월요일, 6: 일요일
df1_clean['is_weekend'] = df1_clean['weekday'].isin([5, 6])  # 토요일, 일요일

def parse_time_slot(ts):
    s = str(ts).replace('시', '')
    if '-' in s:
        start, end = s.split('-')
        return int(start), int(end)
    elif s.isdigit():
        h = int(s)
        return h, h+1   # 단일 시각은 1시간 구간으로
    else:
        return None, None
    
df1_clean[['hour_start', 'hour_end']] = df1_clean['time_slot'].apply(
    lambda x: pd.Series(parse_time_slot(x))
)

def categorize_time_range(start, end):
    """시작/끝 시각 모두 고려해서 시간대 카테고리 분류"""
    if pd.isna(start) or pd.isna(end):
        return '기타'
    mid = (start + end) / 2  # 구간의 중심값 기준
    if 6 <= mid < 9:
        return '출근시간'
    elif 9 <= mid < 12:
        return '오전'
    elif 12 <= mid < 14:
        return '점심시간'
    elif 14 <= mid < 18:
        return '오후'
    elif 18 <= mid < 21:
        return '퇴근시간'
    elif 21 <= mid < 24:
        return '저녁/밤'
    else:
        return '새벽'

df1_clean['time_category'] = df1_clean.apply(
    lambda row: categorize_time_range(row['hour_start'], row['hour_end']),
    axis=1
)

# df2에도 시간 파생 변수 추가
df2_clean['year'] = df2_clean['base_date'].dt.year
df2_clean['month'] = df2_clean['base_date'].dt.month
df2_clean['day'] = df2_clean['base_date'].dt.day
df2_clean['weekday'] = df2_clean['base_date'].dt.weekday
df2_clean['is_weekend'] = df2_clean['weekday'].isin([5, 6])

# df5에도 시간 파생 변수 추가
df5_clean['year'] = df5_clean['base_date'].dt.year
df5_clean['month'] = df5_clean['base_date'].dt.month
df5_clean['day'] = df5_clean['base_date'].dt.day
df5_clean['weekday'] = df5_clean['base_date'].dt.weekday  # 0: 월요일, 6: 일요일
df5_clean['is_weekend'] = df5_clean['weekday'].isin([5, 6])  # 토요일, 일요일

In [23]:
df5_clean.shape

(5613907, 12)

### 4. 중복 제거

In [24]:
# station_name이 같고 total_user_count도 같으면 중복으로 판단하여 하나만 남기고,
# total_user_count가 다르면 별도 행으로 유지
df5_clean = df5_clean.drop_duplicates(
    subset=['station_name', 'total_user_count', 'base_date'],
    keep='first'
).reset_index(drop=True)

# 결과 확인
print("중복 제거 후 shape:", df5_clean.shape)
print(df5_clean.head())

중복 제거 후 shape: (2941593, 12)
   base_date                                       station_name  \
0 2019-01-01  (구)중앙파출소                                      ...   
1 2019-01-01  1100고지휴게소                                     ...   
2 2019-01-01  1100도로입구                                      ...   
3 2019-01-01  6호광장                                          ...   
4 2019-01-01  911의원                                         ...   

   total_user_count   station_id   longitude   latitude location_info  year  \
0               102  406000324.0  126.560894  33.247598   윤성현내과의원 건너편  2019   
1                54  406001894.0  126.462942  33.357820          정보없음  2019   
2                11  406000529.0  126.431520  33.251640          정보없음  2019   
3                20  405000724.0  126.549231  33.510988       스타침대 우측  2019   
4                 7  405001360.0  126.678082  33.540685    아지트게스트하우스앞  2019   

   month  day  weekday  is_weekend  
0      1    1        1       False  
1      1    1      

### 5. 데이터 확인

In [25]:
df1_clean

,base_date,day_of_week,time_slot,user_type,user_count,year,month,day,weekday,is_weekend,hour_start,hour_end,time_category
0,2019-01-01,화,22시-24시,경로,2,2019,1,1,1,False,22,24,저녁/밤
1,2019-01-01,화,22시-24시,어린이,1,2019,1,1,1,False,22,24,저녁/밤
2,2019-01-01,화,22시-24시,일반,25,2019,1,1,1,False,22,24,저녁/밤
3,2019-01-01,화,22시-24시,장애 동반,1,2019,1,1,1,False,22,24,저녁/밤
4,2019-01-01,화,22시-24시,청소년,5,2019,1,1,1,False,22,24,저녁/밤
...,...,...,...,...,...,...,...,...,...,...,...,...,...
272206,2024-10-06,일,22,청소년,125,2024,10,6,6,True,22,23,저녁/밤
272207,2024-10-06,일,23,경로,1,2024,10,6,6,True,23,24,저녁/밤
272208,2024-10-06,일,23,일반,33,2024,10,6,6,True,23,24,저녁/밤
272209,2024-10-06,일,23,장애 일반,1,2024,10,6,6,True,23,24,저녁/밤


In [26]:
df2_clean

,base_date,act_type,bus_number,user_type,user_count,year,month,day,weekday,is_weekend
0,2019-01-01,승차,101-1,어린이,1,2019,1,1,1,False
1,2019-01-01,승차,101-1,일반,53,2019,1,1,1,False
2,2019-01-01,승차,101-1,청소년,15,2019,1,1,1,False
3,2019-01-01,승차,101-2,일반,46,2019,1,1,1,False
4,2019-01-01,승차,101-2,청소년,7,2019,1,1,1,False
...,...,...,...,...,...,...,...,...,...,...
11997303,2024-10-06,환승,820-6,일반,2,2024,10,6,6,True
11997304,2024-10-06,환승,880-1,일반,2,2024,10,6,6,True
11997305,2024-10-06,환승,880-1,청소년,1,2024,10,6,6,True
11997306,2024-10-06,환승,999-9,일반,2,2024,10,6,6,True


In [27]:
df5_clean

,base_date,station_name,total_user_count,station_id,longitude,latitude,location_info,year,month,day,weekday,is_weekend
0,2019-01-01,(구)중앙파출소 ...,102,406000324.0,126.560894,33.247598,윤성현내과의원 건너편,2019,1,1,1,False
1,2019-01-01,1100고지휴게소 ...,54,406001894.0,126.462942,33.357820,정보없음,2019,1,1,1,False
2,2019-01-01,1100도로입구 ...,11,406000529.0,126.431520,33.251640,정보없음,2019,1,1,1,False
3,2019-01-01,6호광장 ...,20,405000724.0,126.549231,33.510988,스타침대 우측,2019,1,1,1,False
4,2019-01-01,911의원 ...,7,405001360.0,126.678082,33.540685,아지트게스트하우스앞,2019,1,1,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2941588,2024-10-06,효성빌라 ...,12,406000170.0,126.576764,33.251648,CU편의점 좌측,2024,10,6,6,True
2941589,2024-10-06,흙담솔사가 ...,1,406001667.0,126.552402,33.258853,정보없음,2024,10,6,6,True
2941590,2024-10-06,흙통 ...,21,406000295.0,126.582277,33.261910,탐라주유소 건너편,2024,10,6,6,True
2941591,2024-10-06,흥국사 ...,7,405001115.0,126.377740,33.452319,용흥3길이정표앞,2024,10,6,6,True


## 미니 프로젝트 2

In [28]:
import matplotlib.pyplot as plt
import seaborn as sns

# 한글 폰트 설정
import matplotlib
matplotlib.rc('font', family='Malgun Gothic')   # Windows
matplotlib.rcParams['axes.unicode_minus'] = False

### 1. 노선별 이용 패턴 분석

In [29]:
def analyze_route_patterns(df2_clean):
    """노선별 총 이용객 수 및 승/하차 패턴 분석"""
    print("=" * 50)
    print("1. 노선별 이용 패턴 분석")
    print("=" * 50)
    
    # 1-1. 노선별 총 이용객 수
    route_total = df2_clean.groupby('bus_number')['user_count'].sum().reset_index()
    route_total = route_total.sort_values('user_count', ascending=False)
    
    print("📊 상위 10개 노선별 총 이용객 수:")
    print(route_total.head(10))
    print()
    
    # 1-2. 노선별 승차/하차/환승 패턴
    route_act_pattern = df2_clean.groupby(['bus_number', 'act_type'])['user_count'].sum().unstack(fill_value=0)
    route_act_pattern['total'] = route_act_pattern.sum(axis=1)
    route_act_pattern = route_act_pattern.sort_values('total', ascending=False)
    
    print("📊 상위 5개 노선별 승차/하차/환승 패턴:")
    print(route_act_pattern.head())
    print()
    
    # 1-3. 노선별 사용자 유형별 이용 현황
    route_user_pattern = df2_clean.groupby(['bus_number', 'user_type'])['user_count'].sum().unstack(fill_value=0)
    route_user_pattern['total'] = route_user_pattern.sum(axis=1)
    route_user_pattern = route_user_pattern.sort_values('total', ascending=False)
    
    print("📊 상위 5개 노선별 사용자 유형별 이용 현황:")
    print(route_user_pattern.head())
    print()
    
    # return route_total, route_act_pattern, route_user_pattern

analyze_route_patterns(df2_clean)

1. 노선별 이용 패턴 분석
📊 상위 10개 노선별 총 이용객 수:
    bus_number  user_count
507     365-21      737994
508     365-22      737019
493      360-1      684723
499      360-2      667841
257      281-2      653569
674      510-1      646655
256      281-1      646097
681      510-2      611573
129     202-14      592943
110     201-14      591566

📊 상위 5개 노선별 승차/하차/환승 패턴:
act_type        승차      하차     환승   total
bus_number                               
365-21      347933  299157  90904  737994
365-22      350578  299265  87176  737019
360-1       319750  273347  91626  684723
360-2       306587  266760  94494  667841
281-2       313876  261012  78681  653569

📊 상위 5개 노선별 사용자 유형별 이용 현황:
user_type      경로    어린이  유공 동반  유공 일반      일반  장애 동반  장애 일반     청소년   total
bus_number                                                                  
365-21      85655  53571    346  32515  286088  64625  66654  148540  737994
365-22      85991  53678    364  31041  287078  64192  66265  148410  737019
360-1    

### 2. 시간대별 이용 패턴 분석

In [30]:
def analyze_time_patterns(df1_clean):
    """시간대별 이용 패턴 분석"""
    print("=" * 50)
    print("2. 시간대별 이용 패턴 분석")
    print("=" * 50)
    
    # 2-1. 시간대별 총 이용객 수 (hour_start 기준)
    hourly_pattern = df1_clean.groupby('hour_start')['user_count'].sum().reset_index()
    hourly_pattern = hourly_pattern.sort_values('hour_start')
    
    print("📊 시간대별 총 이용객 수:")
    print(hourly_pattern)
    print()
    
    # 2-2. 시간 카테고리별 이용 패턴
    time_category_pattern = df1_clean.groupby('time_category')['user_count'].sum().reset_index()
    time_category_pattern = time_category_pattern.sort_values('user_count', ascending=False)
    
    print("📊 시간 카테고리별 이용 패턴:")
    print(time_category_pattern)
    print()
    
    # 2-3. 요일별 시간대별 이용 패턴 (hour_start 기준)
    weekday_hour_pattern = df1_clean.groupby(['weekday', 'hour_start'])['user_count'].sum().unstack(fill_value=0)
    
    print("📊 요일별 시간대별 이용 패턴 (상위 5시간대):")
    top_hours = hourly_pattern.nlargest(5, 'user_count')['hour_start'].tolist()
    print(f"상위 이용 시간대: {top_hours}")
    print(weekday_hour_pattern[top_hours])
    print()
    
    # 2-4. 주말/평일별 시간대별 이용 패턴 (hour_start 기준)
    weekend_hour_pattern = df1_clean.groupby(['is_weekend', 'hour_start'])['user_count'].sum().unstack(fill_value=0)
    
    print("📊 주말/평일별 시간대별 이용 패턴:")
    print(weekend_hour_pattern)
    print()
    
    # return hourly_pattern, time_category_pattern, weekday_hour_pattern, weekend_hour_pattern

analyze_time_patterns(df1_clean)

2. 시간대별 이용 패턴 분석
📊 시간대별 총 이용객 수:
    hour_start  user_count
0            0        6578
1            1     2439684
2            4           3
3            5       29219
4            6      337756
5            7    13214344
6            8      628884
7            9      551507
8           10    12737926
9           11      588617
10          12      598562
11          13    13743735
12          14      607140
13          15      673233
14          16    15017547
15          17      685464
16          18      624197
17          19     9434709
18          20      415329
19          21      342177
20          22     1803250
21          23       25222

📊 시간 카테고리별 이용 패턴:
  time_category  user_count
2            오후    30149293
5          출근시간    14180984
1            오전    13878050
6          퇴근시간    10474235
0            새벽     2475484
3          저녁/밤     2170649
4          점심시간     1176388

📊 요일별 시간대별 이용 패턴 (상위 5시간대):
상위 이용 시간대: [16, 13, 7, 10, 19]
hour_start       16       13       7       

### 3. 사용자 유형별 이용 패턴 분석

In [31]:
def analyze_user_type_patterns(df1_clean, df2_clean):
    """사용자 유형별 이용 패턴 분석"""
    print("=" * 50)
    print("3. 사용자 유형별 이용 패턴 분석")
    print("=" * 50)
    
    # 3-1. df1_clean에서 사용자 유형별 시간대별 이용 패턴
    user_time_pattern = df1_clean.groupby(['user_type', 'hour_start'])['user_count'].sum().unstack(fill_value=0)
    
    print("📊 사용자 유형별 시간대별 이용 패턴 (6-24시):")
    display_hours = list(range(6, 25))  # 6시부터 24시까지
    available_hours = [h for h in display_hours if h in user_time_pattern.columns]
    print(user_time_pattern[available_hours])
    print()
    
    # 3-2. df2_clean에서 사용자 유형별 승차/하차/환승 패턴
    user_act_pattern = df2_clean.groupby(['user_type', 'act_type'])['user_count'].sum().unstack(fill_value=0)
    
    print("📊 사용자 유형별 승차/하차/환승 패턴:")
    print(user_act_pattern)
    print()
    
    # 3-3. 사용자 유형별 주말/평일 이용 패턴
    user_weekend_pattern = df1_clean.groupby(['user_type', 'is_weekend'])['user_count'].sum().unstack(fill_value=0)
    user_weekend_pattern.columns = ['평일', '주말']
    user_weekend_pattern['주말_평일_비율'] = user_weekend_pattern['주말'] / user_weekend_pattern['평일']
    
    print("📊 사용자 유형별 주말/평일 이용 패턴:")
    print(user_weekend_pattern)
    print()
    
    # return user_time_pattern, user_act_pattern, user_weekend_pattern

analyze_user_type_patterns(df1_clean, df2_clean)

3. 사용자 유형별 이용 패턴 분석
📊 사용자 유형별 시간대별 이용 패턴 (6-24시):
hour_start      6        7       8       9        10      11      12       13  \
user_type                                                                       
경로           83637  2646584  122678  117632  2576538  111592  109231  2372034   
어린이           2922   660072   44231   16339   455025   25192   31716   988936   
유공 동반           75     3675     241     295     3908     359     315     2383   
유공 일반         4067   304127   11688   12853   380695   12851   12383   338608   
일반          149076  5290395  241005  241669  5426210  266247  261897  5631310   
장애 동반        26745  1107155   52888   45019   973603   41395   42081   956801   
장애 일반        26743  1142935   53574   50762  1166855   48823   47155  1063493   
청소년          44491  2059401  102579   66938  1755092   82158   93784  2390170   

hour_start      14      15       16      17      18       19      20      21  \
user_type                                                  

### 4. 정류장별 이용 패턴 분석

In [32]:
from datetime import datetime

def analyze_station_patterns(df5_clean, top_n=20):
    """
    정류장별(Station ID 기준) 이용 패턴 분석
    반환: 딕셔너리(여러 요약 테이블)
    """
    print("="*60)
    print("정류장별 이용 패턴 분석 (station_id 중심)")
    print("="*60)

    df = df5_clean.copy()

    # 0) 기본 전처리: 날짜 타입 및 YM 파생
    if not pd.api.types.is_datetime64_any_dtype(df['base_date']):
        # df5_clean이 이미 'YYYY-MM-DD' 문자열이면 format 지정 없이 변환 가능
        df['base_date'] = pd.to_datetime(df['base_date'], errors='coerce')
    df['ym'] = df['base_date'].dt.to_period('M').astype(str)  # 'YYYY-MM'

    # 1) 일자별-정류장별 총합(안전 재집계)
    #    (혹시 동일 station_id, 동일 날짜가 중복 존재해도 합쳐서 안정화)
    station_daily = (
        df.groupby(['station_id', 'station_name', 'base_date', 'is_weekend'], as_index=False)['total_user_count']
          .sum()
    )

    # 2) 정류장별 전체 이용량 TOP N
    station_total = (
        station_daily.groupby(['station_id', 'station_name'], as_index=False)['total_user_count']
                     .sum()
                     .sort_values('total_user_count', ascending=False)
    )
    print("📊 정류장별 전체 이용량 TOP N")
    print(station_total.head(top_n))
    print()

    # 3) 정류장별 '평균 일 이용량' TOP N
    station_avg_daily = (
        station_daily.groupby(['station_id', 'station_name'], as_index=False)['total_user_count']
                     .mean()
                     .rename(columns={'total_user_count': 'avg_daily_users'})
                     .sort_values('avg_daily_users', ascending=False)
    )
    print("📊 정류장별 평균 '일' 이용량 TOP N")
    print(station_avg_daily.head(top_n))
    print()

    # 4) 정류장별 주말/평일 이용 패턴 (합계 & 비율)
    #    주의: is_weekend == True/False → ['평일','주말']로 이름 매핑
    ww = (
        station_daily.groupby(['station_id', 'station_name', 'is_weekend'])['total_user_count']
                     .sum()
                     .unstack(fill_value=0)
    )
    # 컬럼 이름 정리 (False->평일, True->주말)
    if ww.columns.tolist() == [False, True]:
        ww.columns = ['평일', '주말']
    elif ww.columns.tolist() == [True, False]:
        ww = ww[[False, True]]
        ww.columns = ['평일', '주말']
    else:
        # 혹시 is_weekend 값이 한쪽만 존재할 수 있으니 대비
        if False not in ww.columns: ww[False] = 0
        if True not in ww.columns: ww[True] = 0
        ww = ww[[False, True]]
        ww.columns = ['평일', '주말']

    ww['주말_평일_비율'] = (ww['주말'] / ww['평일']).replace([pd.NA, pd.NaT, float('inf')], 0)
    ww = ww.reset_index()

    print("📊 정류장별 주말/평일 이용 패턴 (합계 및 주말/평일 비율)")
    print(ww.sort_values('주말_평일_비율', ascending=False).head(top_n))
    print()

    # 5) 정류장별 요일 패턴 (0=월 ~ 6=일)
    #    weekday 컬럼이 df5_clean에 있으므로 이를 사용 (station_daily로부터 복원)
    #    station_daily에는 weekday 없음 → 원본 df에서 재집계
    station_weekday = (
        df.groupby(['station_id', 'station_name', 'weekday'], as_index=False)['total_user_count']
          .sum()
    )
    weekday_pivot = (
        station_weekday.pivot_table(index=['station_id', 'station_name'],
                                    columns='weekday',
                                    values='total_user_count',
                                    fill_value=0)
    )
    # 보기 좋게 열 이름을 문자열로 치환(월~일)
    weekday_map = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
    weekday_pivot.rename(columns=weekday_map, inplace=True)
    weekday_pivot = weekday_pivot.reset_index()

    print("📊 정류장별 요일 패턴 (합계, 열: 월~일)")
    print(weekday_pivot.head(top_n))
    print()

    # 6) 정류장별 월별 패턴 (YYYY-MM)
    station_monthly = (
        df.groupby(['station_id', 'station_name', 'ym'], as_index=False)['total_user_count']
          .sum()
    )
    monthly_pivot = (
        station_monthly.pivot_table(index=['station_id', 'station_name'],
                                    columns='ym',
                                    values='total_user_count',
                                    fill_value=0)
                       .reset_index()
    )
    # 최근 12개월만 보고 싶으면 다음 라인 주석 해제
    # recent_cols = sorted([c for c in monthly_pivot.columns if c not in ['station_id','station_name']])[-12:]
    # monthly_pivot = monthly_pivot[['station_id','station_name'] + recent_cols]

    print("📊 정류장별 월별 패턴 (합계, 열: YYYY-MM)")
    print(monthly_pivot.head(top_n))
    print()

    # 7) 정류장별 '최대 피크 일자' (언제 가장 많이 이용되었나)
    peak_idx = (
        station_daily.sort_values(['station_id', 'total_user_count'], ascending=[True, False])
                     .groupby('station_id', as_index=False)
                     .first()[['station_id', 'station_name', 'base_date', 'total_user_count']]
                     .rename(columns={'base_date': 'peak_date', 'total_user_count': 'peak_users'})
    )
    print("📊 정류장별 최대 피크 일자/이용량")
    print(peak_idx.head(top_n))
    print()

    # 8) 좌표 포함 종합 요약 (총합/평균/피크)
    #    df에서 좌표/메타 1행을 붙여 요약 테이블 구성
    coords_meta = (
        df.groupby(['station_id', 'station_name'], as_index=False)
          .agg({'longitude':'mean', 'latitude':'mean', 'location_info':'first'})
    )
    summary = (
        station_total.rename(columns={'total_user_count':'total_users'})
        .merge(station_avg_daily, on=['station_id', 'station_name'], how='left')
        .merge(peak_idx, on=['station_id', 'station_name'], how='left')
        .merge(coords_meta, on=['station_id', 'station_name'], how='left')
        .merge(ww, on=['station_id', 'station_name'], how='left')
        .sort_values('total_users', ascending=False)
    )
    # 정리
    summary = summary[['station_id', 'station_name', 'longitude', 'latitude', 'location_info',
                       'total_users', 'avg_daily_users', 'peak_date', 'peak_users', '평일', '주말', '주말_평일_비율']]

    print("📊 종합 요약(좌표/메타 포함): 총합, 평균일이용, 피크일, 주말·평일")
    print(summary.head(top_n))
    print()

    # # 반환(필요시 다음 단계에서 시각화/저장)
    # return {
    #     'station_daily': station_daily,           # 일별-정류장별 합계
    #     'station_total': station_total,           # 정류장별 총합
    #     'station_avg_daily': station_avg_daily,   # 정류장별 평균 일 이용
    #     'weekend_weekday': ww,                    # 주말/평일 합계 및 비율
    #     'weekday_pivot': weekday_pivot,           # 요일 패턴 피벗(월~일)
    #     'monthly_pivot': monthly_pivot,           # 월별 패턴 피벗(YYYY-MM)
    #     'peak_by_station': peak_idx,              # 최대 피크 일자/이용량
    #     'summary': summary                        # 좌표 포함 종합 요약
    # }

results = analyze_station_patterns(df5_clean, top_n=20)
results

정류장별 이용 패턴 분석 (station_id 중심)
📊 정류장별 전체 이용량 TOP N
       station_id                                       station_name  \
165   405000368.0  한라병원                                          ...   
382   405000991.0  제주버스터미널                                       ...   
221   405000484.0  제주동중학교                                        ...   
136   405000301.0  천수동                                           ...   
1010  405002558.0  화북남문                                          ...   
127   405000284.0  동광양                                           ...   
129   405000290.0  동문로터리,동문시장                                    ...   
128   405000288.0  인화초등학교                                        ...   
151   405000335.0  관덕정                                           ...   
83    405000208.0  제주대학교                                         ...   
803   405002036.0  제주시청(광양방면)                                    ...   
802   405002035.0  제주시청(아라방면)                                    ...   
81    40500020

### 5. 종합 통계 및 패턴 식별

In [33]:
def identify_patterns(df1_clean, df2_clean, df5_clean, top_n=5):
    # ---------------------------
    # [A] df1_clean / df2_clean 기반 (사용자 유형 + 노선)
    # ---------------------------

    # 4-1. 피크 시간대 식별
    hourly_total = df1_clean.groupby('hour_start')['user_count'].sum()
    peak_hours = hourly_total.nlargest(3).index.tolist()
    print("📈 피크 이용 시간대")
    for h in peak_hours:
        print(f"   - {h}시: {hourly_total[h]:,}명")
    print()

    # 4-2. 가장 많이 이용되는 노선
    route_usage = df2_clean.groupby('bus_number')['user_count'].sum()
    top_routes = route_usage.nlargest(3)
    print("🚌 가장 많이 이용되는 노선")
    for route, count in top_routes.items():
        print(f"   - {route}: {count:,}명")
    print()

    # 4-3. 사용자 유형별 이용 비율
    user_type_total = df1_clean.groupby('user_type')['user_count'].sum()
    user_type_pct = (user_type_total / user_type_total.sum() * 100).round(1)
    print("👥 사용자 유형별 이용 비율")
    for user_type, pct in user_type_pct.items():
        print(f"   - {user_type}: {pct}% ({user_type_total[user_type]:,}명)")
    print()

    # 4-4. 주말 vs 평일 이용 패턴
    weekend_usage = df1_clean.groupby('is_weekend')['user_count'].sum()
    print("📅 주말 vs 평일 이용 현황")
    print(f"   - 평일: {weekend_usage[False]:,}명 ({weekend_usage[False]/weekend_usage.sum()*100:.1f}%)")
    print(f"   - 주말: {weekend_usage[True]:,}명 ({weekend_usage[True]/weekend_usage.sum()*100:.1f}%)")
    print()

    # 4-5. 연도별 월평균 이용객 수
    yearly_sum = df1_clean.groupby('year')['user_count'].sum()
    yearly_months = df1_clean.groupby('year')['month'].nunique()
    yearly_avg = (yearly_sum / yearly_months).round(0)
    print("📊 연도별 월 평균 이용객 수")
    for year, avg in yearly_avg.items():
        print(f"   - {year}년: 월평균 {avg:,.0f}명")
    print()

    # ---------------------------
    # [B] df5_clean 기반 (정류장 패턴)
    # ---------------------------

    print("=" * 60)
    print("정류장 기반 주요 패턴")
    print("=" * 60)

    # 4-6. 가장 많이 이용된 정류장 (TOP N)
    station_total = df5_clean.groupby(['station_id', 'station_name'])['total_user_count'].sum()
    top_stations = station_total.nlargest(top_n)
    print(f"🚏 가장 많이 이용된 정류장 TOP {top_n}")
    for station, count in top_stations.items():
        sid, sname = station
        print(f"- {sname}(ID {sid}): {count:,}명")  # ← 공백 최소화
    print()

    # 4-7. 정류장별 주말/평일 비율 TOP N
    weekend_pattern = (
        df5_clean.groupby(['station_id','station_name','is_weekend'])['total_user_count']
                 .sum()
                 .unstack(fill_value=0)
    )
    if False in weekend_pattern.columns and True in weekend_pattern.columns:
        weekend_pattern['주말_평일_비율'] = (weekend_pattern[True] / weekend_pattern[False]).replace([float('inf')], 0)
        top_weekend_bias = weekend_pattern.sort_values('주말_평일_비율', ascending=False).head(top_n)
        print(f"🎢 주말 이용 비중이 높은 정류장 TOP {top_n}")
        for idx, row in top_weekend_bias.iterrows():
            sid, sname = idx
            print(f"- {sname}(ID {sid}): 평일 {row[False]:,}명 vs 주말 {row[True]:,}명 (비율 {row['주말_평일_비율']:.2f})")
    print()

    # 4-8. 월별 이용 추세 (최근 연도별 비교)
    monthly_trend = (
        df5_clean.groupby(['year','month'])['total_user_count']
                 .sum()
                 .reset_index()
    )
    print("📊 월별 이용 추세 (연도별 비교)")
    for year in sorted(monthly_trend['year'].unique()):
        year_data = monthly_trend[monthly_trend['year']==year]
        total = year_data['total_user_count'].sum()
        print(f"- {year}년: 총 {total:,}명, 평균 {total/len(year_data):,.0f}명/월")
    print()

    # 4-9. 최대 피크 일자 (정류장별 TOP 1)
    peak_day = (
        df5_clean.groupby(['station_id','station_name','base_date'])['total_user_count']
                 .sum()
                 .reset_index()
                 .sort_values(['station_id','total_user_count'], ascending=[True, False])
                 .groupby('station_id')
                 .first()
                 .reset_index()
    )
    print("🌟 정류장별 최대 피크 일자 TOP N")
    for _, row in peak_day.head(top_n).iterrows():
        print(f"   - {row['station_name']}(ID {row['station_id']}): {row['base_date'].date()}에 {row['total_user_count']:,}명")
    print()

    # ---------------------------
    # [C] 종합 인사이트 요약
    # ---------------------------
    print("=" * 60)
    print("🔎 종합 인사이트 요약")
    print("=" * 60)
    print(f"- 전체적으로 피크 시간대는 {peak_hours}시대에 집중됨")
    print(f"- 가장 많이 이용된 노선: {', '.join(top_routes.index)}")
    print(f"- 전체 이용자 중 {user_type_pct.idxmax()} 비중이 가장 큼({user_type_pct.max()}%)")    
    print(f"- 주말 비중이 가장 높은 정류장: {top_weekend_bias.index[0][1]} (비율 {top_weekend_bias['주말_평일_비율'].iloc[0]:.2f})")
    print(f"- 연도별 월평균 이용객: {dict(yearly_avg)}")
    print()
    
    # return {
    #     "peak_hours": peak_hours,
    #     "top_routes": top_routes,
    #     "user_type_pct": user_type_pct,
    #     "weekend_usage": weekend_usage,
    #     "monthly_trend": monthly_trend,
    #     "top_stations": top_stations,
    #     "top_weekend_bias": top_weekend_bias,
    #     "peak_day": peak_day
    # }

identify_patterns(df1_clean, df2_clean, df5_clean, top_n=5)

📈 피크 이용 시간대
   - 16시: 15,017,547명
   - 13시: 13,743,735명
   - 7시: 13,214,344명

🚌 가장 많이 이용되는 노선
   - 365-21: 737,994명
   - 365-22: 737,019명
   - 360-1: 684,723명

👥 사용자 유형별 이용 비율
   - 경로: 16.5% (12,271,932명)
   - 어린이: 5.5% (4,070,053명)
   - 유공 동반: 0.0% (14,226명)
   - 유공 일반: 2.0% (1,498,343명)
   - 일반: 43.4% (32,332,470명)
   - 장애 동반: 6.9% (5,108,025명)
   - 장애 일반: 7.3% (5,457,682명)
   - 청소년: 18.5% (13,752,352명)

📅 주말 vs 평일 이용 현황
   - 평일: 55,463,841명 (74.4%)
   - 주말: 19,041,242명 (25.6%)

📊 연도별 월 평균 이용객 수
   - 2019년: 월평균 1,392,338명
   - 2020년: 월평균 1,162,877명
   - 2021년: 월평균 1,131,088명
   - 2022년: 월평균 972,942명
   - 2023년: 월평균 957,706명
   - 2024년: 월평균 904,757명

정류장 기반 주요 패턴
🚏 가장 많이 이용된 정류장 TOP 5
- 한라병원                                                                                            (ID 405000368.0): 1,112,113명
- 제주버스터미널                                                                                      (ID 405000991.0): 1,035,281명
- 제주동중학교                                              

## 미니 프로젝트 3

In [34]:
import folium
from folium.plugins import HeatMap

# 1. 정류장별 전체 혼잡도 데이터 준비
station_total = df5_clean.groupby(
    ['station_id', 'station_name', 'longitude', 'latitude']
)['total_user_count'].sum().reset_index()

# 2. Folium 지도 생성
jeju_center = [33.361666, 126.551944]
m = folium.Map(location=jeju_center, zoom_start=10)

# 3. 전체 혼잡도 히트맵 추가
heat_data_total = [
    [row['latitude'], row['longitude'], row['total_user_count']]
    for _, row in station_total.iterrows()
]
HeatMap(heat_data_total, radius=12, name='전체 혼잡도', gradient={0.4: 'blue', 0.65: 'lime', 1: 'red'}).add_to(m)

# 4. 혼잡 상위 정류장 마커 추가 (예: 한라병원, 제주버스터미널)
top_stations = station_total.sort_values('total_user_count', ascending=False).head(20)
for _, row in top_stations.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['station_name']} (ID {row['station_id']})\n총 이용객수: {row['total_user_count']:,}명",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# 5. 레이어 컨트롤 추가
folium.LayerControl().add_to(m)

# 6. 지도 저장
m.save('jeju_bus_congestion_heatmap.html')
print("혼잡도 히트맵이 'jeju_bus_congestion_heatmap.html'로 저장되었습니다.")

혼잡도 히트맵이 'jeju_bus_congestion_heatmap.html'로 저장되었습니다.


In [35]:
df1_clean.to_csv('clean_data/df1_clean.csv', index=False)
df2_clean.to_csv('clean_data/df2_clean.csv', index=False)
df5_clean.to_csv('clean_data/df5_clean.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'clean_data'